<a href="https://colab.research.google.com/github/OmerRosen/Kaggle/blob/main/IMDB_2022_Hit_or_Flop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IMDB 2022 - Hit or Flop

## Data Collection

### Data Requirments

# Feature collection mission

# Main list Page:
* Movie Id
* Movie Title
* Movie year
* Rating
* MetaScore - Outcome
* Description
* Poster
* Directore name + Link
* Stars
* Votes - Output
* Genre

# Movie Page:
* Movie Length
* Rank
* Writer
* Star #1
* Star #2
* Star #3
* User eviews
* Critic reviews
* Number of photos posted
* Storyline - Text
* Tag line
* Release Date - Month
* Country of origin
* Additional Lanaguages
* Country of filming
* Production companies
* Budget
* Opening Weekend Date
* Gross US & Canada - Output
* Opening weekend US & Canada - Output
* Gross worldwide - Output
* Color - Color
* Color - Black&White
* Sound mix - Dolby Digital
* Sound mix - Dolby Atmos
* Aspect ratio

# Director Page:
* Director Age
* Is Top 500?
* Director Gender (Based on bio)
* Previous film count - As Director
* Previous film count - As Writer
* Previous film count - As As Producer
* Director Publicity listing count

# Information Abount Cast:
* Full list of cast and their profile links
* Number of cast members
* Produced by - Is top 500?
* Music by - Is top 500?
* Cinematography by  - Is top 500
* Film Editing by   - Is top 500
* Art Direction by  - Is top 500
* Number of Production Management
* Number of Art Department
* Number of Sound Department
* Number of Camera and Electrical Department
* Number of Editorial Department
* Number of Music Department
* Number of Additional Crew

# Actor Page
* Is Top 5000?
* Is Top 500?
* Is Top 100 (aka - Has numberical rank)
* Is Top 10 (aka - Has numberical rank)
* Numerical Rank (Could be none)
* Gender
* Age
* Oscar nominations 
* Birth country

# Production COmpanies (Filter)
* Num of previous titles


# Aggragated data:
* How many male stars
* How many female stars
* Avrage cast memeber age
* Max/Min age of cast member
* Number of unique birth countries of actors
* Total number of Oscar numinatior for cast
* Num cast members in top 5000
* Num cast memebers in top 500
* Num cast members in top 100
* Num cast members in top 10

# IMDB Scraper

In [ ]:
import requests
from IPython.core.display import HTML
import json
from time import sleep
import re
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import os
import time

In [ ]:
base_url = "https://www.imdb.com"

start_date = "2022-01-01"
end_date = "2022-12-31"
minimum_votes = 10 # Minimum vote amount to coolect movie
start_point = 1 # Start from movie #1-50

movie_search_url = f"{base_url}/search/title/?title_type=feature&release_date={start_date},{end_date}&num_votes={minimum_votes},&{start_point}anguages=en&start=1&ref_=adv_nxt"
movie_search_url

'https://www.imdb.com/search/title/?title_type=feature&release_date=2022-01-01,2022-12-31&num_votes=10,&languages=en&start=1&ref_=adv_nxt'

In [ ]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36"
}

list_of_50_movies = requests.get(f'{movie_search_url}', headers=headers, timeout=10)
list_of_50_movies

<Response [200]>

In [ ]:
list_of_50_movies_soup = BeautifulSoup(list_of_50_movies.text, 'html.parser').find_all('div',{'class':'lister-item mode-advanced'})


In [ ]:
movie_search_page = list_of_50_movies_soup[0]

In [ ]:
def get_basic_details(search_box_info):
  search_box_info = {}

  search_box_info['movie_id'] = movie_search_page.find('span',{'class':'lister-item-index unbold text-primary'}).text.replace('.','')
  search_box_info['movie_name'] = movie_search_page.find('a')
  search_box_info['movie_page_url'] = f"{base_url}{movie_search_page.find('a')['href']}"
  search_box_info['movie_name'] = movie_search_page.find('a').find('img')['alt'].strip()
  search_box_info['movie_thubmnail'] = movie_search_page.find('a').find('img')['src']
  search_box_info['movie_year'] = movie_search_page.find('span',{'class':'lister-item-year text-muted unbold'}).text.replace('(','').replace(')','')
  search_box_info['movie_rating'] = movie_search_page.find('strong').text
  search_box_info['movie_metascore'] = movie_search_page.find('span',{'class':'metascore'}).text.strip()
  search_box_info['movie_description'] = movie_search_page.find_all('p',{'class':'text-muted'})[1].text.strip()
  search_box_info['runtime_min'] = movie_search_page.find('span',{'class':'runtime'}).text.split(' ')[0]

  bottom_box_info = movie_search_page.find('p',{'class':'sort-num_votes-visible'}).find_all('span')
  search_box_info['movie_vote_num'] = bottom_box_info[1]['data-value']
  search_box_info['movie_gross'] = bottom_box_info[4]['data-value'] if len(bottom_box_info)>2 else None;

  search_box_info['movie_rating'] = movie_search_page.find_all('p',{'class':'text-muted'})[0].find('span',{'class':'certificate'}).text.strip()
  search_box_info['movie_genere'] = movie_search_page.find_all('p',{'class':'text-muted'})[0].find('span',{'class':'genre'}).text.strip()

  return search_box_info

In [ ]:
movie_search_page = list_of_50_movies_soup[0]
search_box_info = get_basic_details(movie_search_page)
search_box_info



{'movie_id': '1',
 'movie_name': 'Everything Everywhere All at Once',
 'movie_page_url': 'https://www.imdb.com/title/tt6710474/?ref_=adv_li_i',
 'movie_thubmnail': 'https://m.media-amazon.com/images/S/sash/4FyxwxECzL-U1J8.png',
 'movie_year': '2022',
 'movie_rating': 'R',
 'movie_metascore': '81',
 'movie_description': 'A middle-aged Chinese immigrant is swept up into an insane adventure in which she alone can save existence by exploring other universes and connecting with the lives she could have led.',
 'runtime_min': '139',
 'movie_vote_num': '407835',
 'movie_gross': '72,861,480',
 'movie_genere': 'Action, Adventure, Comedy'}

In [ ]:
movie_main_page = requests.get(search_box_info['movie_page_url'], headers=headers)
movie_main_page_soup = BeautifulSoup(movie_main_page.text, 'html.parser')

In [ ]:
# Movie Page:
#* Movie Length
#* Rank
#* Writer
#* Star #1
#* Star #2
#* Star #3
#* User eviews
#* Critic reviews
#* Storyline - Text -- Loads late
#* Release Date - Month
#* Country of origin
#* Additional Lanaguages
#* Production companies
#* Budget
#* Opening Weekend Date
#* Gross US & Canada - Output
#* Opening weekend US & Canada - Output
#* Gross worldwide - Output
#* Color - Color
#* Color - Black&White
#* Sound mix - Dolby Digital
#* Sound mix - Dolby Atmos
#* Aspect ratio

SyntaxError: ignored

In [ ]:
def get_basic_details(movie_main_page_soup):
  movie_page_dict = {}

  director_box = movie_main_page_soup.find('ul',{'class':'ipc-metadata-list ipc-metadata-list--dividers-all title-pc-list ipc-metadata-list--baseAlt'}).find('span',string="Directors").find_parent()
  movie_page_dict['director_name'] = director_box.find('a').text
  movie_page_dict['director_url'] = f"{base_url}{director_box.find('a')['href']}"
  movie_page_dict['director_id'] = movie_page_dict['director_url'].split('/')[-2]

  writers_box = movie_main_page_soup.find('ul',{'class':'ipc-metadata-list ipc-metadata-list--dividers-all title-pc-list ipc-metadata-list--baseAlt'}).find('span',string="Writers").find_parent()
  movie_page_dict['writer_name'] = director_box.find('a').text
  movie_page_dict['writer_url'] = f"{base_url}{writers_box.find('a')['href']}"
  movie_page_dict['writer_id'] = movie_page_dict['writer_url'].split('/')[-2]

  stars_box = movie_main_page_soup.find('a',{'class':'ipc-metadata-list-item__label ipc-metadata-list-item__label--link'}).find_parent().find_all('a')
  for i,star_box in enumerate(stars_box[1:-1]):
    star_num = f'star_{i+1}'
    movie_page_dict[f'{star_num}_name'] = star_box.text.strip()
    movie_page_dict[f'{star_num}_url'] = f"{base_url}{star_box['href']}"
    movie_page_dict[f'{star_num}_id'] = star_box['href'].split('/')[-2]

  review_scores = movie_main_page_soup.find_all('span',{'class':'score'})
  movie_page_dict['user_reviews_count'] = review_scores[0].text
  movie_page_dict['critic_reviews_count'] = review_scores[1].text

  movie_page_dict['release_date'] = movie_main_page_soup.find('a',{'class':'ipc-metadata-list-item__label ipc-metadata-list-item__label--link'}, string="Release date").find_parent().find('a',{'class':'ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link'}).text.split('(')[0].strip()
  movie_page_dict['release_date']



  movie_page_dict['budget'] = movie_main_page_soup.find('span',{'class':'ipc-metadata-list-item__label'}, string="Budget").find_next_sibling().find('span',{'class':'ipc-metadata-list-item__list-content-item'}).text.split('(')[0].strip()

  movie_page_dict['gross_worldwide'] = movie_main_page_soup.find('span',{'class':'ipc-metadata-list-item__label'}, string="Gross worldwide").find_next_sibling().find('span',{'class':'ipc-metadata-list-item__list-content-item'}).text.split('(')[0].strip()

  movie_page_dict['gross_us_canada'] = movie_main_page_soup.find('span',{'class':'ipc-metadata-list-item__label'}, string="Gross US & Canada").find_next_sibling().find('span',{'class':'ipc-metadata-list-item__list-content-item'}).text.split('(')[0].strip()

  movie_page_dict['opening_weekend_us_canada'] = movie_main_page_soup.find('span',{'class':'ipc-metadata-list-item__label'}, string="Opening weekend US & Canada").find_next_sibling().find('span',{'class':'ipc-metadata-list-item__list-content-item'}).text.split('(')[0].strip()

  movie_page_dict['origin_country'] = movie_main_page_soup.find('span',{'class':'ipc-metadata-list-item__label'}, string="Country of origin").find_next_sibling().find('a').text.split('(')[0].strip()

  languages = movie_main_page_soup.find('span',{'class':'ipc-metadata-list-item__label'}, string="Languages").find_next_sibling().find_all('a')
  movie_page_dict['languages'] = ",".join([language.text for language in languages])

  return movie_page_dict

get_basic_details(movie_main_page_soup)

{'director_name': 'Daniel Kwan',
 'director_url': 'https://www.imdb.com/name/nm3453283/?ref_=tt_ov_dr',
 'director_id': 'nm3453283',
 'writer_name': 'Daniel Kwan',
 'writer_url': 'https://www.imdb.com/name/nm3453283/?ref_=tt_ov_wr',
 'writer_id': 'nm3453283',
 'star_1_name': 'Michelle Yeoh',
 'star_1_url': 'https://www.imdb.com/name/nm0000706/?ref_=tt_ov_st',
 'star_1_id': 'nm0000706',
 'star_2_name': 'Stephanie Hsu',
 'star_2_url': 'https://www.imdb.com/name/nm3513533/?ref_=tt_ov_st',
 'star_2_id': 'nm3513533',
 'star_3_name': 'Jamie Lee Curtis',
 'star_3_url': 'https://www.imdb.com/name/nm0000130/?ref_=tt_ov_st',
 'star_3_id': 'nm0000130',
 'user_reviews_count': '3.3K',
 'critic_reviews_count': '394',
 'release_date': 'April 8, 2022',
 'budget': '$14,300,000',
 'gross_worldwide': '$135,383,773',
 'gross_us_canada': '$76,676,502',
 'opening_weekend_us_canada': '$501,305',
 'origin_country': 'United States',
 'languages': 'English,Mandarin,Cantonese'}